# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Output File (CSV)
cities_df = pd.read_csv("../output_data/Weatherpy.csv")
cities_df = cities_df.drop(['Unnamed: 0'], axis=1)
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Saskylakh,0,RU,1578238656,95,71.97,114.09,-20.90,10.65
1,Mataura,100,NZ,1578238697,88,-46.19,168.86,51.01,15.99
2,Aginskoye,100,RU,1578238999,90,51.11,114.54,11.80,4.14
3,Mount Gambier,5,AU,1578238803,87,-37.83,140.78,48.20,8.05
4,Ulladulla,71,AU,1578238999,100,-35.36,150.48,62.60,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
#Save lat and lng as a dataframe
locations = cities_df[['Lat','Lng']].astype(float)
#Store humidity as array
humidity = cities_df['Humidity'].astype(float)
#Set Humidity as weight
max_humidity = humidity.max()

In [6]:
#Customize the size of the figure
figure_layout = {
    'width': '700px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


In [7]:
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=max_humidity, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
narrowed_df = cities_df.loc[(cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness']== 0) ]
narrowed_df = narrowed_df.dropna(how='any')
narrowed_df = narrowed_df.reset_index()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Create a open list hotel
hotel_df = []
#Set the base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#Set up a parameters dictionary
params = {
    'types': 'Hotel',
    'keyword': 'Lodging',
    'radius': '5000',
    'key': g_key
}

#Use iterrows to iterate through the pandas dataframe
for index, row in narrowed_df.iterrows():
    #Get location from df 
    lat =row["Lat"]
    lng = row["Lng"]
    #Update parameters dict. with lat. & long. info from the df 
    params["location"] = f"{lat},{lng}"
    #Assemebly url and make API request
    print(f'Retrieving Results for Index {index}: {lat}, {lng}.')
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotel_df.append(results[0]['name'])
        narrowed_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result.....skipping.")
        
              
print('-----------------') 
    
                 

Retrieving Results for Index 0: -21.57, 165.5.
Retrieving Results for Index 1: 6.75, -11.37.
Retrieving Results for Index 2: 8.0, 124.29.
Retrieving Results for Index 3: 21.46, -71.14.
Retrieving Results for Index 4: -28.77, 32.06.
Retrieving Results for Index 5: 15.12, 104.33.
Retrieving Results for Index 6: 18.74, 7.39.
Retrieving Results for Index 7: -33.69, -53.46.
Retrieving Results for Index 8: 14.89, 105.88.
Retrieving Results for Index 9: 11.28, -15.83.
-----------------


In [15]:
narrowed_city_df = narrowed_df.dropna(how='any')
narrowed_city_df = narrowed_city_df.drop(['index'], axis=1)
hotel_df = narrowed_city_df

In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…